In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

### 合并操作 
- pd.merge(left, right)

In [2]:
# 创建员工表 emp(id, name, age, dept_id)
# 创建部门表 dept (dept_id, dept_name)
# 创建管理者 manager(dept_id, supervisor_id)
dept = DataFrame({
    'dept_id': ['1001', '1002', '1003'],
    'dept_name': ['软件部', '会计部', '生产部'],
    
})

emp = DataFrame({
    'id': ['01', '02', '03', '04', '05'],
    'name': ['disen', 'lucy', 'jack', 'judy', 'mack'],
    'age': [16, 17, 20, 19, 21],
    'dept_id': ['1001', '1002', '1001', '1001', '1003'],
})

In [3]:
dept

,dept_id,dept_name
0,1001,软件部
1,1002,会计部
2,1003,生产部


In [4]:
emp

,id,name,age,dept_id
0,01,disen,16,1001
1,02,lucy,17,1002
2,03,jack,20,1001
3,04,judy,19,1001
4,05,mack,21,1003


In [5]:
# 将部门表和员工表进行合并
pd.merge(emp, dept)

,id,name,age,dept_id,dept_name
0,01,disen,16,1001,软件部
1,03,jack,20,1001,软件部
2,04,judy,19,1001,软件部
3,02,lucy,17,1002,会计部
4,05,mack,21,1003,生产部


In [10]:
# 添加新的员工, 部门是新成立：1004部门
emp = emp.append(Series({
    'id': '06',
    'name': 'jerry',
    'age': 33,
    'dept_id': '1004'
}, name=5))  # append(Series) 增加一行数据时，Series中必须指定行索引标签

In [11]:
emp

,id,name,age,dept_id
0,01,disen,16,1001
1,02,lucy,17,1002
2,03,jack,20,1001
3,04,judy,19,1001
4,05,mack,21,1003
5,06,jerry,33,1004


In [13]:
# how 指定的合并方式， {'inner', 'outer', 'left', 'right'}
# inner 两个表都存在的数据，进行合并， 类似数据库的内连接
# outer 两个表的数据全部显示，不管是否都存在, 类似数据库的全外连接
# left 除了共有的数据之外，额外将左表中数据也显示
# right 除了共有的数据之外，额外将右表中的数据也显示
# 注： outer/left/right都可能会出现Nan值
pd.merge(emp, dept, how='outer')

,id,name,age,dept_id,dept_name
0,01,disen,16,1001,软件部
1,03,jack,20,1001,软件部
2,04,judy,19,1001,软件部
3,02,lucy,17,1002,会计部
4,05,mack,21,1003,生产部
5,06,jerry,33,1004,NaN


In [14]:
dept = dept.append({
    'dept_id': '1005',
    'dept_name': '测试部'
}, ignore_index=True)  # ignore_index=True重新创建行索引
dept

,dept_id,dept_name
0,1001,软件部
1,1002,会计部
2,1003,生产部
3,1005,测试部


In [15]:
pd.merge(emp, dept, how='outer')

,id,name,age,dept_id,dept_name
0,01,disen,16.0,1001,软件部
1,03,jack,20.0,1001,软件部
2,04,judy,19.0,1001,软件部
3,02,lucy,17.0,1002,会计部
4,05,mack,21.0,1003,生产部
5,06,jerry,33.0,1004,NaN
6,NaN,NaN,NaN,1005,测试部


In [18]:
manager = DataFrame({
    'manage_id': ['01', '02', '03'],  # 管理层的员工的id
    'department_id':   ['1001', '1001', '1003'], # 部门的id
    'mgr_name':  ['Rose', 'Kobe', 'Oleay']
})
manager

,manage_id,department_id,mgr_name
0,01,1001,Rose
1,02,1001,Kobe
2,03,1003,Oleay


In [21]:
# 将员工表和管理者的表进行合并
# 如果两个表合并时没有相同的列标签，则使用left_on和right_on 指定两个表连接字段
# 如果两个表中出现了相同列标签超出2个以上，则使用on指定使用哪一个字段进行连接
pd.merge(emp, manager, left_on='dept_id', right_on='department_id')

,id,name,age,dept_id,manage_id,mgr_name
0,01,disen,16,1001,01,Rose
1,01,disen,16,1001,02,Kobe
2,03,jack,20,1001,01,Rose
3,03,jack,20,1001,02,Kobe
4,04,judy,19,1001,01,Rose
5,04,judy,19,1001,02,Kobe
6,05,mack,21,1003,03,Oleay


In [23]:
emp.shape

(6, 4)

In [29]:
# 扩展drop(index=, columns=) 删除行或列
# 修改字段名
manager.rename(columns={'department_id': 'dept_id',
                        'manage_id': 'mgr_id'}, inplace=True)
manager

,mgr_id,dept_id,mgr_name
0,01,1001,Rose
1,02,1001,Kobe
2,03,1003,Oleay


In [24]:
# 增加员工表的 mgr_id
emp['mgr_id'] = Series(['01', '01', '02', '03', '01', '02'], 
                       index=emp.index)
emp

,id,name,age,dept_id,mgr_id
0,01,disen,16,1001,01
1,02,lucy,17,1002,01
2,03,jack,20,1001,02
3,04,judy,19,1001,03
4,05,mack,21,1003,01
5,06,jerry,33,1004,02


In [32]:
# on指定两个表使用哪一列进行合并
pd.merge(emp, manager, on='dept_id', suffixes=['_北京', '_西安'])

,id,name,age,dept_id,mgr_id_北京,mgr_id_西安,mgr_name
0,01,disen,16,1001,01,01,Rose
1,01,disen,16,1001,01,02,Kobe
2,03,jack,20,1001,02,01,Rose
3,03,jack,20,1001,02,02,Kobe
4,04,judy,19,1001,03,01,Rose
5,04,judy,19,1001,03,02,Kobe
6,05,mack,21,1003,01,03,Oleay
